In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [14]:
#### K = number of countries = number of capitals
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []

        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
        for pair in pairs:
            temp3 = npr.uniform()
            
            if temp3 <= 0.5:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])
            elif temp3 <= 0.75:
                sentence.append(countries[pair])
            else:
                sentence.append(capitals[pair])
                
        sentence += list(np.random.choice(randoms, L - len(sentence), replace = False))

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [15]:
def get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, S, L, q1, q2, embed_dim, n_sentences)

    acc_countries = []
    prob_countries = []

    for _ in range(n_samples):
        sentence = []
        random_countries = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_country in random_countries:
            sentence.append('country_' + str(random_country))
            sentence.append('capital_' + str(random_country))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_countries[-1])]
        acc_countries.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_countries.append(temp[0][vocab_map['capital_' + str(random_countries[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_countries), np.mean(prob_countries))

In [16]:
K = 10 # number of countries
L = 8 # sentence length
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [17]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 4.02201e-34)
(0.0, 2.3161344e-32)
(0.0, 4.0661887e-38)
(0.0, 1.4104819e-37)
(0.0, 8.6619975e-34)
(0.0, 3.1690252e-32)
(0.0, 2.888169e-31)
(0.0, 1.8353992e-33)
(0.0, 6.487085e-38)
(0.0, 1.8004736e-36)
(0.0, 3.467743364152655e-32)


In [18]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 0.001220165)
(0.0, 0.0012740527)
(0.0, 0.0012170984)
(0.0, 0.0011657353)
(0.0, 0.0009795047)
(0.0, 0.0010941147)
(0.0, 0.0012769657)
(0.0, 0.0011780419)
(0.0, 0.0010779926)
(0.0, 0.0011299934)
(0.0, 0.0011613664450123907)


In [19]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 6.0728855e-31)
(0.0, 3.3580639e-31)
(0.0, 4.714825e-15)
(0.0, 2.7262625e-29)
(0.0, 4.166895e-31)
(0.0, 2.6478285e-18)
(0.0, 4.2192183e-18)
(0.0, 4.55418e-18)
(0.0, 5.3733585e-30)
(0.0, 4.306775e-29)
(0.0, 4.726246321773037e-16)


In [20]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.46577448)
(1.0, 0.47689477)
(1.0, 0.45475355)
(1.0, 0.4545888)
(1.0, 0.47462305)
(1.0, 0.4558761)
(1.0, 0.46711951)
(1.0, 0.47491962)
(1.0, 0.46705177)
(1.0, 0.47733486)
(0.9999999999999999, 0.46689365208148953)


In [21]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(0.0, 0.028662467)
(0.0, 0.030035693)
(0.0, 0.028578553)
(0.0, 0.025884686)
(0.0, 0.024634361)
(0.0, 0.02804324)
(0.0, 0.025818123)
(0.0, 0.026257604)
(0.0, 0.028830184)
(0.0, 0.027264695428311825)


In [22]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs = 0
probs = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c \
        = get_acc_prob(K, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    
    accs += acc_c[0]/10
    probs += acc_c[1]/10
    
print((accs, probs))

(1.0, 0.24131641)
(1.0, 0.23955533)
(1.0, 0.2558268)
(1.0, 0.22745374)
(1.0, 0.26036778)
(1.0, 0.25960225)
(1.0, 0.23885767)
(1.0, 0.24991575)
(1.0, 0.25016883)
(1.0, 0.24761637)
(0.9999999999999999, 0.2470680922269821)
